**Tester le code avec des événements plus probable!!!**
## To do list:
* Q1-1
#####* Splitting (Diane) (faudrait jouer avec le nombre de niveaux de splitting, donc avec la valeur de la variable "seuil", pour voir ce qui donne la meilleure précision)
#####* Echantillonage d'importance (Peng-Wei) (C'est bon!!, à tester différent f)
#####* Chaîne de Markov
#####* Graphique : Comparaison de différentes méthodes.

* Q1-2
#####* Quantile processus AR(1) (comme dans calcul liste_a Q1-1) (Diane)
#####* Quantile échantillonnage d'importance (Diane)
* Q2
#####* M-C (Peng-Wei)

* Q3
# Finished :
* Q1
#####* Monte-Carlo naïve
* Q


In [1]:
import numpy as np
import math
import matplotlib.pyplot
## Calcul de IC pour les quantiles
from scipy.stats import binom
from math import ceil
def IC_quantile(quantiles, niveau):
    n = len(quantiles)
    quantiles = np.sort(quantiles)
    index_ = ceil(n * niveau) - 1
    niveau_inf = ceil(n * (niveau - 1.96 * np.sqrt(niveau * (1-niveau) / n))) - 1
    niveau_sup = ceil(n * (niveau + 1.96 * np.sqrt(niveau * (1-niveau) / n))) - 1
    q = quantiles[index_]
    print("Quantile au niveau {} = {}, avec intervalle de confiance = {}, {}".format(niveau, 
                                                                                     q, 
                                                                                     quantiles[niveau_inf], 
                                                                                     quantiles[niveau_sup]))

## Q1

Dans cette première modélisation simplifiée, on considère donc que le prix $P_t$ est un processus de Poisson de paramètres $\lambda, \nu$ où $\nu$ est la loi des incréments $J_n$. 

Pour un temps d'attente moyen entre deux sauts de $300s$, on prend $\lambda = \dfrac{1}{300}$


In [2]:
# On identifie le processus par le processus de Poisson composé, qui finit à un temps fixé T
# On crée des fonctions pour modéliser le processus.

#### On définit nu, la loi des incréments
# Ancien : plus lent
saut_1_ancien = lambda x: np.random.choice([-1, 1], size=x, replace=True, p=[0.5, 0.5]) #correspond à m=1
saut_2_ancien = lambda x: np.random.choice([-3, -2, -1, 1, 2, 3], size=x, replace=True, p=0.5*np.array([1/6, 1/3, 1/2, 1/2, 1/3, 1/6])) #correspond à m=3

# Nouveau : beaucoup plus vite
value_1 = np.array([-1, 1])
value_2 = np.array([-3, -2, -2, -1, -1, -1, 1, 1, 1, 2, 2, 3])
saut_1 = lambda x : value_1[np.random.randint(low=2, size=x)]
saut_2 = lambda x : value_2[np.random.randint(low=12, size=x)]

# Les paramètres
P0 = 35
T = 4*60*60
lamb = 1/300

### Q2-2 Quantile

#### MCMC Naif

In [3]:
#Naif - Méthode 1


def Quantile_naif_2(seuil,n,lamb1,lamb2,T,P0,saut):

    liste_P_t = np.zeros(n)
    for i in range(n):
        N1 = np.random.poisson(lamb1*T)
        N2 = np.random.poisson(lamb2*T)
        P2 = 0 
        if (N2%2!=0): #si N2 est pair, la contribution du processus 2 vaut 0
            P2=-1+2*np.random.binomial(1,0.5)
        increments = saut(N1)
        
        liste_P_t[i] = np.sum(increments)+P0 +P2
    liste_P_t = np.sort(liste_P_t)
    return liste_P_t[int(np.ceil(n*seuil))-1]

#### MCMC numba

In [4]:
#MC Accéléré - Méthode 2 (pas plus rapide ?)

# On identifie le processus par le processus de Poisson composé, qui finit à un temps fixé T
# On crée des fonctions pour modéliser le processus.


def Q2_2_MC(P0, T, lamb1, lamb2, saut):
  # saut : une fonction qui prend la taille de sortie comme entrée pour les sauts.
  N1 = np.random.poisson(lam=lamb1 * T)
  T1 = np.sort(np.random.uniform(low=0.0, high=T, size=N1))
  J1 = saut(N1)

  N2 = np.random.poisson(lam=lamb2 * T)
  T2 = np.sort(np.random.uniform(low=0.0, high=T, size=N2))

  J2_0 = saut_1(1)[0]

  somme = P0
  index_T2 = 0
  for i in range(len(T1)):
    while index_T2 < N2 and T2[index_T2] < T1[i]:
      somme += J2_0
      J2_0 *= -1
      index_T2 += 1
    somme += J1[i]
  return somme                      # 1 échantillon

def Q2_2_MC_n(P0, T, lamb1, lamb2, saut, size):
  res = []
  for i in range(size):
    res.append(Q2_2_MC(P0, T, lamb1, lamb2, saut))
  return np.array(res)

def quantile(echantillion, size, quant):
  index_bot = int(math.ceil(size * quant))
  index_top = int(math.ceil(size * (1 - quant)))
  echantillion.partition((index_bot, index_top))
  return echantillion[index_bot], echantillion[index_top]

def Question2_2_naif(size, quant, saut, P0, T, lamb1, lamb2):
  return quantile(Q2_2_MC_n(P0, T, lamb1, lamb2, saut, size), size, quant)

In [5]:
%%time 
P0 = 35
T = 4*60*60
lamb1 = 1/660
lamb2 = 1/110
n = int(1e7)
quantiles = Q2_2_MC_n(P0, T, lamb1, lamb2, saut_1, n)
for quantile in [1e-6, 1e-5, 1e-4, 1 - 1e-4, 1 - 1e-5, 1 - 1e-6]:
    IC_quantile(quantiles, quantile)
#(19, 52) pour m=1 seuil 10^-4 n =10^4
#(14,56) pour m=1 seuil 10^-5 n =10^5
#Donne ... pour n = 10^6 et seuil=1e-6 et m=1
#Donne (0, 69) pour n = 10^4 et seuil=1e-4 et m=3
#Donne (-2, 71) pour n = 10^5 et seuil=1e-5 et m=3
#Donne pour n = 10^5 et seuil=1e-6 et m=3

Quantile au niveau 1e-06 = 12, avec intervalle de confiance = 11, 12
Quantile au niveau 1e-05 = 14, avec intervalle de confiance = 14, 15
Quantile au niveau 0.0001 = 17, avec intervalle de confiance = 17, 17
Quantile au niveau 0.9999 = 53, avec intervalle de confiance = 53, 53
Quantile au niveau 0.99999 = 56, avec intervalle de confiance = 55, 56
Quantile au niveau 0.999999 = 58, avec intervalle de confiance = 57, 59
CPU times: user 21min 47s, sys: 3.24 s, total: 21min 50s
Wall time: 21min 54s


In [6]:
%%time
quantiles = Q2_2_MC_n(P0, T, lamb1, lamb2, saut_2, n)
for quantile in [1e-6, 1e-5, 1e-4, 1 - 1e-4, 1 - 1e-5, 1 - 1e-6]:
    IC_quantile(quantiles, quantile)
#Donne 17/55 pour n = 10^4 et seuil=1e-4 et m=1
#Donne (12.0, 56.0) pour n = 10^5 et seuil=1e-5 et m=1
#Donne (11.0, 58.0) pour n = 10^6 et seuil=1e-6 et m=1
#Donne (2.0, 65.0) pour n = 10^4 et seuil=1e-4 et m=3
#Donne (-4.0, 73.0) pour n = 10^5 et seuil=1e-5 et m=3
#Donne (-10.0, 77.0) pour n = 10^6 et seuil=1e-6 et m=3

Quantile au niveau 1e-06 = -8, avec intervalle de confiance = -10, -7
Quantile au niveau 1e-05 = -3, avec intervalle de confiance = -3, -3
Quantile au niveau 0.0001 = 2, avec intervalle de confiance = 2, 2
Quantile au niveau 0.9999 = 68, avec intervalle de confiance = 68, 68
Quantile au niveau 0.99999 = 73, avec intervalle de confiance = 72, 73
Quantile au niveau 0.999999 = 77, avec intervalle de confiance = 76, 78
CPU times: user 21min 32s, sys: 2.47 s, total: 21min 35s
Wall time: 21min 40s


### Echant d'imp

In [7]:
from numpy.random import exponential, choice
from numba import jit

@jit(nopython=True)
def rand_choice_nb(arr, prob):
    """
    :param arr: A 1D numpy array of values to sample from.
    :param prob: A 1D numpy array of probabilities for the given samples.
    :return: A random sample from the given array with a given probability.
    """
    return arr[np.searchsorted(np.cumsum(prob), np.random.random(), side="right")]

@jit(nopython=True)
def quantile_sample(lamb1,lamb2,T,P0,esp, values, p):
    #returns 0,0 if no price is strictly negative
    #returns 1, 1/L_T otherwise
    t1=exponential(1/lamb1/esp)
    t2=exponential(1/lamb2)
    P = P0
    sumIncrements1=0
    increment2 = -1+2*np.random.binomial(1,0.5) #pour savoir si le premier incrément du processus 2 vaut 1 ou -1
    while t1<T or t2<T:
        if (t2<t1):
            P+=increment2
            increment2 *=-1
            t2+=exponential(1/lamb2)
        else:
            increment1=rand_choice_nb(values,p)
            sumIncrements1+=increment1
            P+=increment1
            t1+=exponential(1/lamb1/esp)
    return P,np.exp(lamb1*T*(esp-1)-c*sumIncrements1)

def echant_imp(M,lamb1,lamb2,T,P0,c,m):
    liste_quantiles = np.zeros((M, 2))
    if m == 3:
        esp = np.cosh(c)/2 +np.cosh(2*c)/3 + np.cosh(3*c)/6
        values = np.array([-3,-2,-1,1,2,3])
        p = np.array([np.exp(-3*c)/12,np.exp(-2*c)/6,np.exp(-c)/4,np.exp(c)/4,np.exp(2*c)/6,np.exp(3*c)/12])/esp
    else:
        esp = np.cosh(c)
        values = np.array([-1, 1])
        p = np.array([np.exp(-c),np.exp(c)])/(2*np.cosh(c))
    from tqdm.notebook import tqdm
    for l in tqdm(range(M)):
        liste_quantiles[l] = quantile_sample(lamb1,lamb2,T,P0,esp,values,p)

    liste_quantiles = liste_quantiles[(liste_quantiles[:, 0]).argsort()]             # sort by P_T
    liste_quantiles[:, 1] = liste_quantiles[:, 1].cumsum() / M
    return liste_quantiles

In [8]:
def IC_quantile_importance(quantiles, niveau):
    n = len(quantiles)
    
    index_ = np.where(quantiles[:, 1] > niveau)[0][0]
    niveau_inf = (niveau - 1.96 * np.sqrt(niveau * (1-niveau) / n))
    niveau_sup = (niveau + 1.96 * np.sqrt(niveau * (1-niveau) / n))
    index_inf = np.where(quantiles[:, 1] > niveau_inf)[0][0]
    index_sup = np.where(quantiles[:, 1] > niveau_sup)[0][0]
    q = quantiles[index_]
    print("Quantile au niveau {} = {}, avec intervalle de confiance = {}, {}".format(niveau, 
                                                                                     int(q[0]), 
                                                                                     int(quantiles[index_inf][0]), 
                                                                                     int(quantiles[index_sup][0])))

In [11]:
%%time
M=int(1e7)
c=-1
m=1
P0=35
T=4*60*60
lamb1 = 1/660
lamb2 = 1/110

values_L_T = echant_imp(M,lamb1,lamb2,T,P0,c,m)

quantiles = [1e-4, 1e-5, 1e-6]
for quantile in quantiles:
    IC_quantile_importance(values_L_T, quantile)


Quantile au niveau 0.0001 = 17, avec intervalle de confiance = 17, 17
Quantile au niveau 1e-05 = 14, avec intervalle de confiance = 14, 15
Quantile au niveau 1e-06 = 12, avec intervalle de confiance = 11, 12
CPU times: user 1min 36s, sys: 496 ms, total: 1min 37s
Wall time: 1min 37s


In [12]:
# Recompile
@jit(nopython=True)
def rand_choice_nb(arr, prob):
    """
    :param arr: A 1D numpy array of values to sample from.
    :param prob: A 1D numpy array of probabilities for the given samples.
    :return: A random sample from the given array with a given probability.
    """
    return arr[np.searchsorted(np.cumsum(prob), np.random.random(), side="right")]

@jit(nopython=True)
def quantile_sample(lamb1,lamb2,T,P0,esp, values, p):
    #returns 0,0 if no price is strictly negative
    #returns 1, 1/L_T otherwise
    t1=exponential(1/lamb1/esp)
    t2=exponential(1/lamb2)
    P = P0
    sumIncrements1=0
    increment2 = -1+2*np.random.binomial(1,0.5) #pour savoir si le premier incrément du processus 2 vaut 1 ou -1
    while t1<T or t2<T:
        if (t2<t1):
            P+=increment2
            increment2 *=-1
            t2+=exponential(1/lamb2)
        else:
            increment1=rand_choice_nb(values,p)
            sumIncrements1+=increment1
            P+=increment1
            t1+=exponential(1/lamb1/esp)
    return P,np.exp(lamb1*T*(esp-1)-c*sumIncrements1)

In [13]:
M=int(1e7)
c=-0.3
m=3
P0=35
T=4*60*60
lamb1 = 1/660
lamb2 = 1/110
values_L_T = echant_imp(M,lamb1,lamb2,T,P0,c,m)
for quantile in quantiles:
    IC_quantile_importance(values_L_T, quantile)


Quantile au niveau 0.0001 = 2, avec intervalle de confiance = 2, 2
Quantile au niveau 1e-05 = -3, avec intervalle de confiance = -4, -3
Quantile au niveau 1e-06 = -8, avec intervalle de confiance = -10, -7


## Quantiles à droite- echant d'importance

In [14]:
from numpy.random import exponential, choice
from numba import jit

@jit(nopython=True)
def rand_choice_nb(arr, prob):
    """
    :param arr: A 1D numpy array of values to sample from.
    :param prob: A 1D numpy array of probabilities for the given samples.
    :return: A random sample from the given array with a given probability.
    """
    return arr[np.searchsorted(np.cumsum(prob), np.random.random(), side="right")]

@jit(nopython=True)
def quantile_sample(lamb1,lamb2,T,P0,esp, values, p):
    #returns 0,0 if no price is strictly negative
    #returns 1, 1/L_T otherwise
    t1=exponential(1/lamb1/esp)
    t2=exponential(1/lamb2)
    P = P0
    sumIncrements1=0
    increment2 = -1+2*np.random.binomial(1,0.5) #pour savoir si le premier incrément du processus 2 vaut 1 ou -1
    while t1<T or t2<T:
        if (t2<t1):
            P+=increment2
            increment2 *=-1
            t2+=exponential(1/lamb2)
        else:
            increment1=rand_choice_nb(values,p)
            sumIncrements1+=increment1
            P+=increment1
            t1+=exponential(1/lamb1/esp)
    return P,np.exp(lamb1*T*(esp-1)-c*sumIncrements1)

def echant_imp(M,lamb1,lamb2,T,P0,c,m):
    liste_quantiles = np.zeros((M, 2))
    if m == 3:
        esp = np.cosh(c)/2 +np.cosh(2*c)/3 + np.cosh(3*c)/6
        values = np.array([-3,-2,-1,1,2,3])
        p = np.array([np.exp(-3*c)/12,np.exp(-2*c)/6,np.exp(-c)/4,np.exp(c)/4,np.exp(2*c)/6,np.exp(3*c)/12])/esp
    else:
        esp = np.cosh(c)
        values = np.array([-1, 1])
        p = np.array([np.exp(-c),np.exp(c)])/(2*np.cosh(c))
    from tqdm.notebook import tqdm
    for l in tqdm(range(M)):
        liste_quantiles[l] = quantile_sample(lamb1,lamb2,T,P0,esp,values,p)

    liste_quantiles = liste_quantiles[(-liste_quantiles[:, 0]).argsort()]             # sort by P_T
    liste_quantiles[:, 1] = liste_quantiles[:, 1].cumsum() / M
    return liste_quantiles

In [15]:
%%time
M=int(1e7)
c=1
m=1
P0=35
T=4*60*60
lamb1 = 1/660
lamb2 = 1/110

values_L_T = echant_imp(M,lamb1,lamb2,T,P0,c,m)

quantiles = [1e-4, 1e-5, 1e-6]
for quantile in quantiles:
    IC_quantile_importance(values_L_T, quantile)


Quantile au niveau 0.0001 = 53, avec intervalle de confiance = 53, 53
Quantile au niveau 1e-05 = 56, avec intervalle de confiance = 56, 55
Quantile au niveau 1e-06 = 58, avec intervalle de confiance = 59, 58
CPU times: user 1min 36s, sys: 579 ms, total: 1min 37s
Wall time: 1min 37s


In [16]:
# Recompile
@jit(nopython=True)
def rand_choice_nb(arr, prob):
    """
    :param arr: A 1D numpy array of values to sample from.
    :param prob: A 1D numpy array of probabilities for the given samples.
    :return: A random sample from the given array with a given probability.
    """
    return arr[np.searchsorted(np.cumsum(prob), np.random.random(), side="right")]

@jit(nopython=True)
def quantile_sample(lamb1,lamb2,T,P0,esp, values, p):
    #returns 0,0 if no price is strictly negative
    #returns 1, 1/L_T otherwise
    t1=exponential(1/lamb1/esp)
    t2=exponential(1/lamb2)
    P = P0
    sumIncrements1=0
    increment2 = -1+2*np.random.binomial(1,0.5) #pour savoir si le premier incrément du processus 2 vaut 1 ou -1
    while t1<T or t2<T:
        if (t2<t1):
            P+=increment2
            increment2 *=-1
            t2+=exponential(1/lamb2)
        else:
            increment1=rand_choice_nb(values,p)
            sumIncrements1+=increment1
            P+=increment1
            t1+=exponential(1/lamb1/esp)
    return P,np.exp(lamb1*T*(esp-1)-c*sumIncrements1)

In [17]:
M=int(1e7)
c=0.3
m=3
P0=35
T=4*60*60
lamb1 = 1/660
lamb2 = 1/110
values_L_T = echant_imp(M,lamb1,lamb2,T,P0,c,m)
for quantile in quantiles:
    IC_quantile_importance(values_L_T, quantile)


Quantile au niveau 0.0001 = 68, avec intervalle de confiance = 68, 68
Quantile au niveau 1e-05 = 73, avec intervalle de confiance = 74, 73
Quantile au niveau 1e-06 = 78, avec intervalle de confiance = 80, 77
